In [1]:
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

import os
import h5py
from nltk.util import usage
import numpy as np
import datetime
import time
import warnings

from featureExtraction.tf.tf import *
from featureExtraction.tfidf.tfidf import *
from featureExtraction.fasttext.fasttext import *
from featureExtraction.lexicon.lexicon import *
from featureExtraction.orthography.orthography import *
from featureExtraction.pos.postag import *
from featureExtraction.tf.tf import *
from featureExtraction.word2vec.word2vec import *
from featureExtraction.ngram.ngram import *

from featureSelection.kFold import *

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
clean = [x for x in os.listdir("./Dataset/clean")]
clean.sort()

DATA_CLEAN = ["./Dataset/clean/" + x for x in os.listdir("./Dataset/clean")]
DATA_CLEAN.sort()
len_clean = len(DATA_CLEAN)

### GENERATE MODEL

In [3]:
data = []
for i in DATA_CLEAN:
    df = pd.read_csv(i, delimiter=";", low_memory=False, header=0)
    df.dropna(axis=0, inplace=True)
    for tweet in df["teks"].tolist():
        list_kata = eval(tweet)
        data.append(list_kata)

model_cbow_hs_100 = Word2Vec(data, min_count=1, window=5, hs=1, negative=0, vector_size=100)
model_cbow_hs_100.save('./featureExtraction/word2vec/models_all/model_cbow_hs_100.model')

model_cbow_neg_100 = Word2Vec(data, min_count=1, window=5, hs=0, negative=5, vector_size=100)
model_cbow_neg_100.save('./featureExtraction/word2vec/models_all/model_cbow_neg_100.model')

model_cbow_hs_200 = Word2Vec(data, min_count=1, window=5, hs=1, negative=0, vector_size=200)
model_cbow_hs_200.save('./featureExtraction/word2vec/models_all/model_cbow_hs_200.model')

model_cbow_neg_200 = Word2Vec(data, min_count=1, window=5, hs=0, negative=5, vector_size=200)
model_cbow_neg_200.save('./featureExtraction/word2vec/models_all/model_cbow_neg_200.model')

model_cbow_hs_300 = Word2Vec(data, min_count=1, window=5, hs=1, negative=0, vector_size=300)
model_cbow_hs_300.save('./featureExtraction/word2vec/models_all/model_cbow_hs_300.model')

model_cbow_neg_300 = Word2Vec(data, min_count=1, window=5, hs=0, negative=5, vector_size=300)
model_cbow_neg_300.save('./featureExtraction/word2vec/models_all/model_cbow_neg_300.model')

### Extract Features

In [4]:
def word2vec_cbow(filepath, hs, neg, vector_size):
    df = pd.read_csv(filepath, delimiter=";", low_memory=False, header=0)
    model_path = './featureExtraction/word2vec/models_all/model_cbow_{}_{}.model'.format('hs' if hs == 1 else 'neg', vector_size)
    model_cbow = Word2Vec.load(model_path)

    word2vec_arr=[]
    for row in df['teks'].tolist():
        tweets = eval(row)
        row_mean_vector = (np.mean([model_cbow.wv[terms] for terms in tweets], axis=0)).tolist()
        if not (type(row_mean_vector) is list):
            row_mean_vector = [float(0) for i in range(vector_size)]
        word2vec_arr.append(row_mean_vector)

    return np.array(word2vec_arr), df['label'].tolist()
    


### Runner with Classification

In [10]:
warnings.filterwarnings("ignore")
score_list = []
count = 1
fail = 0
# for i in DATA_CLEAN:
for i in DATA_CLEAN:
    print("Running {}/{}".format(count, len_clean), end=" ")
    try:
        # timer mulai
        result_fe, label = word2vec_cbow(i, 0, 5, 300)

        # create dataset
        # X, y = make_classification(n_samples=len(result_fe), n_features=len(result_fe[0]), random_state=1)
        # prepare the cross-validation procedure

        cv = KFold(n_splits=20, random_state=1, shuffle=True)
        x_train, x_test, y_train, y_test = train_test_split(result_fe, label)
                
        logRes = LogisticRegression(random_state=1, max_iter=1000)
        logRes.fit(x_train, y_train)
        predictions = logRes.predict(x_test)
        
        scores = cross_val_score(logRes, result_fe, label, scoring='f1_micro', cv=cv, n_jobs=-1)
        print('F1-score: %.3f (%.3f)' % (mean(scores), std(scores)))
        score_list.append(mean(scores))
        count += 1
        # timer selesai
    except Exception as e:
        count += 1
        fail += 1
        print(i, ": ", e)
        pass

print((sum(score_list)/len(score_list)))
print("Failed: {}".format(fail))

Running 1/20 F1-score: 0.889 (0.096)
Running 2/20 F1-score: 0.775 (0.089)
Running 3/20 F1-score: 0.925 (0.015)
Running 4/20 F1-score: 0.676 (0.084)
Running 5/20 F1-score: 0.740 (0.023)
Running 6/20 F1-score: 0.891 (0.017)
Running 7/20 F1-score: 0.713 (0.113)
Running 8/20 F1-score: 0.715 (0.105)
Running 9/20 F1-score: 0.649 (0.138)
Running 10/20 F1-score: 0.618 (0.185)
Running 11/20 F1-score: 0.684 (0.087)
Running 12/20 F1-score: 0.795 (0.147)
Running 13/20 F1-score: 0.590 (0.086)
Running 14/20 F1-score: 0.737 (0.129)
Running 15/20 F1-score: 0.665 (0.054)
Running 16/20 F1-score: 0.723 (0.020)
Running 17/20 F1-score: 0.564 (0.033)
Running 18/20 F1-score: 0.564 (0.033)
Running 19/20 F1-score: 0.558 (0.055)
Running 20/20 F1-score: 0.740 (0.065)
0.7103841175239662
Failed: 0
